# Install Tensorflow with GPU Support on Ubuntu

This document aimed for installing Tensorflow with GPU support on ubuntu. All the information comes from [tensorflow installation guide](https://www.tensorflow.org/install/install_linux) and _StackOverflow_ as well as _askUbuntu_.  
I assume you have a machine with [CUDA supported Nvidia GPUs](https://developer.nvidia.com/cuda-gpus) and a linux system (ubuntu for this document) installed.  
If you are just interested in installing cpu version tensorflow, you can jump to _Step 4_. Install GPU Supported Tensorflow_.

Thanks Daniel Reichman for sharing his helpful notes!

## Step 0. Install Drivers for Graphic Card

Please go to [this Tensorflow page](https://www.tensorflow.org/install/install_linux) and [this SO page](https://stackoverflow.com/questions/30820513/what-is-version-of-cuda-for-nvidia-304-125/30820690#30820690) to determine which CUDA & nvidia drivers are supported for the TF version you want to install.

If you have a new computer and haven't installed the driver for graphic card yet, just open **Settings**, then click **Software & Updates**, choose **Additional Drivers** tab, select 'Using NVIDIA binary driver' (I'm using a GTX1080 Ti, ver 384 is the latest driver) then click 'Apply Changes'

![Additional Drivers](./pics/additional_drivers.png)

You'll have driver installed after restart the machine.

If you already know the specific driver version, you can also do 
```bash
sudo apt-mark hold nvidia-[ver number]
```
to make sure you have exactly the version you want.

To verify you have the driver properly installed, use the following command:
```Shell
lspci | grep -i nvidia
```
You shall see something like this:

![card info](./pics/nvidia_info.png)

If you had any problem with the drivers, [this page](http://www.linuxandubuntu.com/home/how-to-install-latest-nvidia-drivers-in-linux) might be helpful. You can also do
```bash
sudo apt-get purge nvidia*
sudo apt-get purge cuda*
sudo apt-get remove libcupti-dev
```
and delete all files to start over.

## Step 1. CUDA® Toolkit 8.0

You can find a detailed (but might not very helpful) guide from [Nvidia's website](http://docs.nvidia.com/cuda/cuda-installation-guide-linux/#axzz4VZnqTJ2A). Here is the gist and some necessary steps not mentioned.

### Requirements:
1. CUDA-capable GPU
2. A supported version of Linux with a gcc compiler and toolchain

### Install CUDA

Go to [CUDA download page](https://developer.nvidia.com/cuda-downloads), and select the following install method:

![cuda install](./pics/cuda_install.png)

All my selections are marked in darker green, you can switch based on your situation, but please make sure you are choosing **deb(local)**. If use runfile, you have to shutdown the [X server](https://askubuntu.com/questions/7881/what-is-the-x-server), which can be annoying.

Tips: You can do `sudo apt-get install [package name] -y` to avoid press "y" for installation.

After downloading the deb file, cd to directory where you put the file, do the following commands:
```Shell
sudo dpkg -i cuda-repo-ubuntu1404_8.0.61-1_amd64.deb
sudo apt-get update
sudo apt-get install cuda
```

The name of the deb file might be different for different cuda (CUDA 8 here) and ubuntu version (14.04 here).

### Add CUDA to Path

You need to add CUDA to **PATH** for setting up the enviornment, just tpye in the following command:
```Shell
export PATH=/usr/local/cuda-8.0/bin${PATH:+:${PATH}}
```

You also need to include lib to **LD_LIBRARY_PATH**:
```Shell
export LD_LIBRARY_PATH=/usr/local/cuda-8.0/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
```
For a 32bit system, use:
```Shell
export LD_LIBRARY_PATH=/usr/local/cuda-8.0/lib${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
```

Now do `source ~/.bashrc` to restart terminal or reboot the computer.

### Verify CUDA Installation

This step **is important** because sometimes installation might fail for a variety of reasons, even if you finished the above steps, I highly recommend you take this step.

1. Install writable samples (these samples can go to anywhere):
```Shell
cuda-install-samples-8.0.sh \dir\to\destination
```
2. Make samples (this may take a long time)
```Shell
cd \dir\to\destination\NVIDIA_CUDA-8.0_Samples
make
```
3. Run an example (now you are in dir _NVIDIA_CUDA-8.0_Samples_)
```Shell
cd 1_Utilities/deviceQuery
./deviceQuery 
```

If CUDA is successfully installed, you are expected to see something like this:

![deviceQuery](./pics/device_query.png)

I have two GTX1080 Ti in my computer, and as you can see both of the GPUs have been detected. You might see something different but corresponding to your GPUs.

If verification fails, run the following command
```bash
sudo dmesg | grep NVRM
```
to get more info and also refer to [this page](https://devtalk.nvidia.com/default/topic/876432/cuda-setup-and-installation/no-cuda-capable-device-is-detected/)

## Step 2. Install cuDNN

Go to [Nvidia's cuDNN page](https://developer.nvidia.com/cudnn) for downloading, you need to sign in first if you don't have an Nvidia account before.

After finishing the registration, agree their terms and select the cuDNN version to download. But be very careful, Tensorflow now only works with **cuDNN v6.0** (though the latest version is v7.02). We just installed CUDA 8, so we'll download cuDNN v6.0 with CUDA 8.

![cudnn](./pics/cudnn.png)

Please notice the links in black blocks are what you need to download. You might download different runtime libarry, developer library and code samples based on your ubuntu version.

The above is true up until now (09/21/2017), make sure you check [tensorflow gpu guide](https://www.tensorflow.org/install/install_linux) first.

### Install Library

Extract the cuDNN archive to a directory of your choice, the path to that directory will be referred as **`<installpath>`** in the following commands. The extracted folder should have a name of **cuda**.

```Shell
cd <installpath>/lib
export LD_LIBRARY_PATH=`pwd`:$LD_LIBRARY_PATH
```

You also need to link the includes and library to your compiler:
```Shell
export CPPFLAGS='-I<installpath>/include'
export LDFLAGS='-L<installpath>/lib64'
```
If you are using a 32 bit system:
```Shell
export CPPFLAGS='-I<installpath>/include'
export LDFLAGS='-L<installpath>/lib'
```

You do not need to do ./configure here.

### Install deb

You need to install **Runtime Library**, **Developer Library** and **Code Samples and User Guide** at this step. You have already downloaded them, just cd to the folder and:
```Shell
sudo dpkg -i libcudnn6_6.0.21-1+cuda8.0_amd64.deb
sudo dpkg -i libcudnn6-dev_6.0.21-1+cuda8.0_amd64.deb 
sudo dpkg -i libcudnn6-doc_6.0.21-1+cuda8.0_amd64.deb 
```

Note the files names might be different

## Step 3.  Install libcupti-dev Library

lipcupti-dev is the NVIDIA CUDA Profile Tools Interface. This step is super easy:
```Shell
sudo apt-get install libcupti-dev
```

## Step 4. Install GPU Supported Tensorflow

There are [many ways to install tensorflow](https://www.tensorflow.org/install/install_linux#determine_how_to_install_tensorflow), but I still recommend installing with anaconda. It can help you maintain seperate virtual environments, which could be extremly useful if you are working on multiple projects and they use diffrent version of python or libraries (or the machine is shared by different students).

### Download Anaconda

Just go to [Anaconda's webpage](https://www.anaconda.com/download/), download the version you would like to use (I'll use Python 3.6 for the rest of this tutorial). Then use the command (assume you put them into Downloads):

```Shell
bash ~/Downloads/Anaconda3-4.4.0-Linux-x86_64.sh
```

If you are using Python 2.7, do:
```Shell
bash ~/Downloads/Anaconda2-4.4.0-Linux-x86_64.sh
```

### Create Conda Virtual Environment

I've made a [yaml file](./scripts/environment.yml) for creating a virtual environment. Just a Python 3.6 version with few frequently used packages, we can add more to this list in the future, so the new student or new computer can get startted much easier.

Go to the folder where you put that **environment.yml** file, and use the command:
```Shell
conda env create
```

After the process finished, you shall have a conda virtual env named 'tf-aml'. You can do:
```Shell
source activate tf-aml
source deactivate
```
To enter or exit the virtual environment. For more instructions of using conda, you can refer to this [cheat sheet](https://conda.io/docs/_downloads/conda-cheatsheet.pdf)

Now let's enter the virtual environment, then use the following command:
```Shell
pip install --ignore-installed --upgrade https://storage.googleapis.com/tensorflow/linux/gpu/tensorflow_gpu-1.3.0-cp36-cp36m-linux_x86_64.whl
```
The link should be the corresponding version you find at [this page](https://www.tensorflow.org/install/install_linux#the_url_of_the_tensorflow_python_package)

Now if everything goes well, you have GPU supported tensorflow installed on your computer.

### Jupyter Notebook & Verify Install

You can test to verify if you have sucessfully installed the tensorflow now by run this [short program](https://www.tensorflow.org/install/install_linux#run_a_short_tensorflow_program). But in this document let's try to do with jupyter notebook.

First activate the **tf-aml** virtual environment, then use the following command
```Shell
conda install nb_conda
```
This will "link" your **tf-aml** jupyter notebook so that the jupyter notebook can find your kernel.

Now type 'jupyter notebook' in command line, a webpage will be open. Go to the folder where you put this file (you can either clone thie repo or download this file to your computer), open this file. You need to change the kernel to tf-aml before running the program.

![change kernel](./pics/change_kernel.png)

Now click the cells below, use **Ctrl+Enter** to execute the cell: 

In [8]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


In [9]:
from tensorflow.python.client import device_lib

local_device_protos = device_lib.list_local_devices()
print([x.physical_device_desc for x in local_device_protos if x.device_type == 'GPU'])

['device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0', 'device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0']


You shall see the first cell print the exact same thing as I have here in this notebook, the second cell will print out your gpu info.

If the above two cells can be executed withour error, then **congradualations you have the GPU supported Tensorflow installed : )**

## Bonus: Setting Up Pycharm's Interpreter

If you are using Pycharm as IDE, you might not know how to use **tf-aml** when running files. It's actually pretty easy, just go to settings, and set the project interpreter to tf-aml.

![pycharm kernel](./pics/pycharm_kernel.png)